In [273]:
# Program to read the .tdms file

#**************** Read multiple pakts in tdms file (Should not use for single pkt data)**************************
#pkt_size=39087
# 10pkts in 1sec
# total samples=390870 in 1 sec
#********************************************

#from operator import index
from nptdms import TdmsFile
import numpy as np
import matplotlib.pyplot as plt


In [290]:
# Define global constants below
data_size=43 # Change nos. of bits in single event

# change index[x], index[y] for single packet reading in line 161

in_file_name="/home/ayushnema/Documents/work_DP2/tdms_conversions/Sandeep_files/TIFR_43bit_czt_pingpong_fcc_pingpong_23-9-2022_test7.tdms"

tdms_file = TdmsFile.read(in_file_name)
with TdmsFile.open(in_file_name) as tdms_file:
        group = tdms_file["Untitled"]
        all_groups = tdms_file.groups()
        print (all_groups)

        channel=group["Untitled"]
        all_group_channels = group.channels()
        print(all_group_channels)
        print("Single channel:",channel)

        all_channel_data = channel[:]
        print(all_channel_data)
        print("******************* Input TDMS file details **********************")
        print("Input file name:",in_file_name)
        print("***************** Total samples details **************************")
        #pkt_size=len(all_channel_data)
        #print(pkt_size)
        print("Total samples in acquired data:",len(all_channel_data)) # print total elemnts in array
        # Iterate through the array
full_pkt_data=[]
header=[]
event_data=[]
#val_data=[]
print(type(all_channel_data))
print(all_channel_data.dtype)  ####################to see the data type , SH
print(group)  ####################to see the data type , SH

[<TdmsGroup with path /'Untitled'>]
[<TdmsChannel with path /'Untitled'/'Untitled'>]
Single channel: <TdmsChannel with path /'Untitled'/'Untitled'>
[1 1 1 ... 0 0 0]
******************* Input TDMS file details **********************
Input file name: /home/ayushnema/Documents/work_DP2/tdms_conversions/Sandeep_files/TIFR_43bit_czt_pingpong_fcc_pingpong_23-9-2022_test7.tdms
***************** Total samples details **************************
Total samples in acquired data: 590000
<class 'numpy.ndarray'>
uint8
<TdmsGroup with path /'Untitled'>


In [275]:
for i in range(0,(len(all_channel_data)),1):
    full_pkt_data.append(all_channel_data[i])
#print(full_pkt_data[293739:293783]) # 293783 means, print (293783-1)
print("Length of full_pkt_data:",len(full_pkt_data))

# find out header index using numpy    
arr1=np.array(full_pkt_data)

header_initial_index=np.where((arr1 == 1) & (np.roll(arr1,-1) == 1) & (np.roll(arr1,-2) == 1) & (np.roll(arr1,-3) == 1) & 
               (np.roll(arr1,-4) == 1) & (np.roll(arr1,-5) == 0) & (np.roll(arr1,-6) == 0) & 
               (np.roll(arr1,-7) == 1)  & (np.roll(arr1,-8) == 1)  & (np.roll(arr1,-9) == 0)
                & (np.roll(arr1,-10) == 1) & (np.roll(arr1,-11) == 0) & (np.roll(arr1,-12) == 0)
                & (np.roll(arr1,-13) == 1) & (np.roll(arr1,-14) == 0) & (np.roll(arr1,-15) == 0)
                & (np.roll(arr1,-16) == 0) & (np.roll(arr1,-17) == 0) & (np.roll(arr1,-18) == 1)) [0]
print("index:",header_initial_index)
print("Length of index:",len(header_initial_index))


Length of full_pkt_data: 590000
index: [     0   5547  11094  16641  22188  27735  33282  38829  44376  49923
  55470  61017  66564  72111  77658  83205  88752  94299  99846 105393
 110940 116487 122034 127581 133128 138675 144222 149769 155316 160863
 166410 171957 177504 183051 188598 194145 199692 205239 210786 216333
 221880 227427 232974 238521 244068 249615 255162 260709 266256 271803
 277350 282897 288444 293991 299538 305085 310632 316179 321726 327273
 332820 338367 343914 349461 355008 360555 366102 371649 377196 382743
 388290 393837 399384 404931 410478 416025 421572 427119 432666 438213
 443760 449307 454854 460401 465948 471495 477042 482589 488136 493683
 499230 504777 510324 515871 521418 526965 532512 538059 543606 549153
 554700 560247 565794 571341 576888 582435 587982]
Length of index: 107


In [276]:
pkt_size= header_initial_index[1]
print(pkt_size)

5547


In [277]:
head_ctr=0
data_ctr=0
val_ctr=0
det_id_ctr=0
pix_energy_ctr=0
timetag_ctr=0
header_ctr=0
pix_id_ctr=0

timetag=[]
det_id=[]
pix_id=[]
pix_energy=[]
det_id_arr=[]
det_id_dec=[]
pix_id_arr=[]
pix_id_dec=[]
pix_energy_arr=[]
pix_energy_dec=[]
timetag_arr=[]
timetag_dec=[]
header_arr=[]
header_dec=[]
header_dec_part2=[]
header_dec_part3=[]

#timetag variables
bit_count=0
bin_weight=19
sum7=0

#det_id variable
bit_count1=0
bin_weight1=4
sum8=0

#pix_id variable
bit_count2=0
bin_weight2=7
sum9=0

#pix_energy variable
bit_count3=0
bin_weight3=9
sum10=0

#header variable
bit_count4=0
bin_weight4=15
sum11=0

timetag_err=[]
det_id_err=[]
pix_id_err=[]
pix_energy_err=[]
header_dec=[]


In [278]:
pkt0=[]

# Logic to not read partial packet data
print("******************** Packet data Report *****************")

total_full_pkt=int(len(all_channel_data)/pkt_size)
#print("total packets in acquired data:",full_pkt_size)


value=total_full_pkt*pkt_size

#print("value:",(total_full_pkt*pkt_size))
#print("value1",(len(all_channel_data)-(value*data_size)))

value1=len(all_channel_data)-(value)
print("samples in partial pkt data:",value1)
#print("total samples in partial packet:",(len(all_channel_data)-((len(all_channel_data))/data_size)*data_size))
full_pkt_size=len(all_channel_data)-value1
print("samples in full_pkt_data:",full_pkt_size)


******************** Packet data Report *****************
samples in partial pkt data: 2018
samples in full_pkt_data: 587982


In [286]:
header_data=[]
event_data=[]
header_indices=[]
event_indices=[]
index=0
header_counter=0

for index in header_initial_index:
    for i in range(0,42):
        header_indices.append(index+i)

#print(header_indices_index)
#print(full_pkt_data)
for index in range(len(full_pkt_data)):
    if index in header_indices:
        header_data.append(full_pkt_data[index])
        header_counter+=1
    else:
        event_data.append(full_pkt_data[index])
        event_indices.append(index)



TypeError: 'int' object is not iterable

In [280]:
print(len(event_data))
print(len(header_indices))
print(len(full_pkt_data))
print(len(event_indices))
print(len(header_data))

585506
4494
590000
585506
4494


In [285]:
#sandeep's output here is 63908 10930 1647
#header into decimal
    #header_data
    #header_indices
    #event_data
    #event_indices
header_data_decimal_1=[]
header_data_decimal_2=[]
header_data_decimal_3=[]
sum3=0
sum5=0
sum6=0

for converter in range (len(header_initial_index)):
    sum3=((32768*header_data[converter])+(16384*(header_data[converter+1]))+(8192*(header_data[converter+2]))+(4096*(header_data[converter+3]))+(2048*(header_data[converter+4]))+(1024*(header_data[converter+5]))+(512*(header_data[converter+6]))+(256*(header_data[converter+7]))+(128*(header_data[converter+8]))+(64*(header_data[converter+9]))+(32*(header_data[converter+10]))+(16*(header_data[converter+11]))+(8*(header_data[converter+12]))+(4*(header_data[converter+13]))+(2*(header_data[converter+14]))+(1*(header_data[converter+15])))
    header_data_decimal_1.append(sum3)
    sum5=((32768*(header_data[converter+16]))+(16384*(header_data[converter+17]))+(8192*(header_data[converter+18]))+(4096*(header_data[converter+19]))+(2048*(header_data[converter+20]))+(1024*(header_data[converter+21]))+(512*(header_data[converter+22]))+(256*(header_data[converter+23]))+(128*(header_data[converter+24]))+(64*(header_data[converter+25]))+(32*(header_data[converter+26]))+(16*(header_data[converter+27]))+(8*(header_data[converter+28]))+(4*(header_data[converter+29]))+(2*(header_data[converter+30]))+(1*(header_data[converter+31])))
    header_data_decimal_2.append(sum5)
    sum6=((1024*(header_data[converter+32]))+(512*(header_data[converter+33]))+(256*(header_data[converter+34]))+(128*(header_data[converter+35]))+(64*(header_data[converter+36]))+(32*(header_data[converter+37]))+(16*(header_data[converter+38]))+(8*(header_data[converter+39]))+(4*(header_data[converter+40]))+(2*(header_data[converter+41]))+(1*(header_data[converter+42])))
    header_data_decimal_3.append(sum6)
    header_ctr=0
    
print(sum3,sum5,sum6)

TypeError: only integer scalar arrays can be converted to a scalar index

In [282]:
# Segregate event_data array into time tag, det. ID, pixel ID and pixel energy array
print(event_data)
for data in event_data:
        if(data_ctr<=19):
                timetag.append(data)
                data_ctr=data_ctr+1
        elif(data_ctr>19 and data_ctr<=24):
                det_id.append(data)
                data_ctr=data_ctr+1
        elif(data_ctr>24 and data_ctr<=32):
                pix_id.append(data)
                data_ctr=data_ctr+1
        elif(data_ctr>32 and data_ctr<=42):
                pix_energy.append(data)
                data_ctr=data_ctr+1
                if(data_ctr==43):
                        data_ctr=0
print("timetag:",timetag[345060:345080])
print("Length: of time tag:",len(timetag))


[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 

In [283]:
#print(det_id)
for det_id_val in det_id:
    if(bit_count1<=4):
        sum8=sum8+(det_id_val*(pow(2,bin_weight1)))
        bin_weight1-=1
        bit_count1+=1
        #print("ele:",element)
        #print(sum)
    if (bit_count1>4 and bit_count1==5):
        #print("ele1:",element)
        det_id_dec.append(sum8)
        bit_count1=0
        bin_weight1=4
        sum8=0
        
#print("timetag_dec:",timetag_dec)
for k in det_id_dec:
    if (k>10 or k<10):
        det_id_err.append(k)
print("det_id error:",det_id_err)
print("Length of det_id error:",len(det_id_err))


det_id error: [30, 11, 14, 17, 20, 23, 27, 14, 1, 4, 7, 11, 14, 17, 20, 4, 14, 11, 14, 17, 20, 23, 27, 14, 17, 20, 23, 27, 30, 1, 4, 20, 14, 27, 30, 1, 4, 7, 11, 30, 1, 4, 7, 11, 14, 17, 20, 4, 30, 11, 14, 17, 20, 23, 27, 14, 17, 20, 23, 27, 30, 1, 4, 20, 14, 27, 30, 1, 4, 7, 11, 30, 1, 4, 7, 11, 14, 17, 20, 4, 4, 12, 4, 28, 16, 12, 4, 29, 16, 12, 4, 28, 4, 28, 8, 12, 1, 12, 0, 28, 17, 20, 0, 28, 0, 14, 0, 30, 4, 14, 0, 28, 8, 12, 8, 28, 0, 14, 8, 28, 16, 12, 0, 28, 0, 12, 0, 28, 31, 5, 7, 8, 11, 13, 7, 0, 2, 3, 5, 7, 8, 2, 7, 5, 7, 8, 11, 13, 7, 8, 11, 13, 15, 16, 18, 7, 13, 15, 16, 18, 19, 21, 15, 16, 18, 19, 21, 23, 24, 26, 18, 15, 21, 23, 24, 26, 27, 29, 23, 24, 26, 27, 29, 31, 0, 2, 26, 23, 29, 31, 0, 2, 3, 5, 31, 0, 2, 3, 5, 7, 8, 2, 27, 25, 13, 19, 19, 17, 14, 3, 26, 3, 12, 1, 22, 3, 0, 3, 23, 5, 12, 3, 19, 17, 8, 3, 29, 21, 12, 3, 18, 18, 12, 3, 22, 17, 8, 3, 19, 1, 14, 7, 2, 1, 12, 3, 20, 5, 14, 19, 15, 18, 19, 20, 21, 21, 22, 19, 0, 1, 1, 2, 3, 4, 5, 1, 19, 2, 3, 4, 5, 5, 6, 

In [284]:
for pix_id_val in pix_id:
    if(bit_count2<=7):
        sum9=sum9+(pix_id_val*(pow(2,bin_weight2)))
        bin_weight2-=1
        bit_count2+=1
        #print("ele:",element)
        #print(sum)
    if (bit_count2>7 and bit_count2==8):
        #print("ele1:",element)
        pix_id_dec.append(sum9)
        bit_count2=0
        bin_weight2=7
        sum9=0            

#print("timetag_dec:",timetag_dec)
for m in pix_id_dec:
    if (m>129 or m<129):
        pix_id_err.append(m)
#print("Length of pix_id error:",len(pix_id_err))
